In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cp -r /content/drive/MyDrive/Class_Vis_Geo/CosPlace /content/drive/MyDrive/Class_Vis_Geo/CosPlace_post_rerank

In [ ]:

# Improve robustness to night domain:
# - Domain adaptation

# General improvements:
# - Re-Ranking / Post-Processing
# - undistortion
# - domain adaptation

# https://docs.google.com/document/d/1qkqQFlQ8z3Mj4Iy_Ckrih8R8PYBzaEPVJGrpmo49NLY/edit#
#  !git clone https://github.com/gmberton/CosPlace


In [3]:
import os
import time
import shutil
!pip install faiss-gpu
!pip install faiss-cpu 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 62.2 MB/s eta 0:00:00


In [4]:
start_time = time.time()

os.chdir('/content')

if  not os.path.isfile('/content/small/train/37.70/@0544204.32@4173406.33@10@S@037.70683@-122.49851@TYcjxIohRl--XFaR4OgdxA@@0@@@@201910@@.jpg'):
  print('data will be extracted in :' ,os.getcwd()) 
  !unzip -q '/content/drive/MyDrive/Class_Vis_Geo/datasets/sf-xs.zip'

  print(time.time() - start_time  )



data will be extracted in : /content
90.21714687347412


In [ ]:
train_file = os.listdir('/content/small/train')
train_size = 0
for i in train_file:
  train_size += len(os.listdir('/content/small/train' +'/'+i))
print((train_size))

59650


In [7]:
len(os.listdir('/content/small/train/37.71'))

4329

In [ ]:
print('sanf val queries :',len(os.listdir('/content/small/val/queries')))
print('sanf val database:',len(os.listdir('/content/small/val/database')))
print('sanf test queries :',len(os.listdir('/content/small/test/queries_v1')))
print('sanf test database:',len(os.listdir('/content/small/test/database')))

sanf val queries : 7993
sanf val database: 8015
sanf test queries : 1000
sanf test database: 27191


In [ ]:
os.chdir('/content/drive/MyDrive/Class_Vis_Geo/CosPlace')
min_images_per_class=10
dataset_name ='small'
M=10
alpha=30
N=5
L=2
filename = f"cache/{dataset_name}_M{M}_N{N}_mipc{min_images_per_class}.torch"
import torch

classes_per_group, images_per_class = torch.load(filename)
print(len(images_per_class))
print(len(classes_per_group[0]))

5965
5965


# 1.Train

In [ ]:
#   GEM has been used in GeoLocalizationNet model 
!python train.py --dataset_folder /content/small --groups_num 1 --epochs_num 10 \
--resume_train /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/default/2022-12-29_14-57-17/last_checkpoint.pth\
--save_dir cos_place_epoch10


2023-01-12 11:37:00   train.py --dataset_folder /content/small --groups_num 1 --epochs_num 10 --resume_train /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/default/2022-12-29_14-57-17/last_checkpoint.pth --save_dir cos_place_epoch10
2023-01-12 11:37:00   Arguments: Namespace(L=2, M=10, N=5, alpha=30, augmentation_device='cuda', backbone='resnet18', batch_size=32, brightness=0.7, classifiers_lr=0.01, contrast=0.7, dataset_folder='/content/small', device='cuda', epochs_num=10, fc_output_dim=512, groups_num=1, hue=0.5, infer_batch_size=16, iterations_per_epoch=10000, lr=1e-05, min_images_per_class=10, num_workers=8, positive_dist_threshold=25, random_resized_crop=0.5, resume_model=None, resume_train='/content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/default/2022-12-29_14-57-17/last_checkpoint.pth', saturation=0.7, save_dir='cos_place_epoch10', seed=0, test_set_folder='/content/small/test', train_set_folder='/content/small/train', use_amp16=False, val_set_folder='/content/small/val')


# 2.evaluation


# sanf

In [ ]:
os.chdir('/content/drive/MyDrive/Class_Vis_Geo/CosPlace')
#  in training defalut backbone (resnet 18 was used)
# in training  fc_output_dim =512
! python3 eval.py --dataset_folder  /content/small --backbone resnet18 \
 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/cos_place_epoch10/2023-01-12_11-37-00/best_model.pth

2023-01-13 19:15:50   eval.py --dataset_folder /content/small --backbone resnet18 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/cos_place_epoch10/2023-01-12_11-37-00/best_model.pth
2023-01-13 19:15:50   Arguments: Namespace(L=2, M=10, N=5, alpha=30, augmentation_device='cuda', backbone='resnet18', batch_size=32, brightness=0.7, classifiers_lr=0.01, contrast=0.7, dataset_folder='/content/small', device='cuda', epochs_num=50, fc_output_dim=512, groups_num=8, hue=0.5, infer_batch_size=16, iterations_per_epoch=10000, lr=1e-05, min_images_per_class=10, num_workers=8, positive_dist_threshold=25, random_resized_crop=0.5, resume_model='/content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/cos_place_epoch10/2023-01-12_11-37-00/best_model.pth', resume_train=None, saturation=0.7, save_dir='default', seed=0, test_set_folder='/content/small/test', use_amp16=False)
2023-01-13 19:15:50   The outputs are being saved in logs/default/2023-01-13_19-15-50
/usr/local/li

## tokyo xs

In [ ]:
os.chdir('/content')

if  not os.path.isfile('/content/tokyo_xs/test/queries/@0381523.71@3946790.15@54@S@035.65786@0139.69112@00685.jpg@@@@@@@@.jpg'):
  print('data will be extracted in :' ,os.getcwd()) 
  !unzip -q '/content/drive/MyDrive/Class_Vis_Geo/datasets/tokyo-xs.zip'

os.chdir("/content/tokyo_xs/test") 
os.rename("queries","queries_v1")


data will be extracted in : /content


In [ ]:
print('tokyo test queries :',len(os.listdir('/content/tokyo_xs/test/queries_v1')))
print('tokyo test database:',len(os.listdir('/content/tokyo_xs/test/database')))

tokyo test queries : 315
tokyo test database: 12771


## checking both saved model 

In [ ]:
os.chdir('/content/drive/MyDrive/Class_Vis_Geo/CosPlace')
#  in training defalut backbone (resnet 18 was used)
# in training  fc_output_dim =512
! python3 eval.py --dataset_folder /content/tokyo_xs --backbone resnet18 \
 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/default/2022-12-30_05-54-55/best_model.pth

2022-12-30 06:30:26   eval.py --dataset_folder /content/tokyo_xs --backbone resnet18 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/default/2022-12-30_05-54-55/best_model.pth
2022-12-30 06:30:26   Arguments: Namespace(L=2, M=10, N=5, alpha=30, augmentation_device='cuda', backbone='resnet18', batch_size=32, brightness=0.7, classifiers_lr=0.01, contrast=0.7, dataset_folder='/content/tokyo_xs', device='cuda', epochs_num=50, fc_output_dim=512, groups_num=8, hue=0.5, infer_batch_size=16, iterations_per_epoch=10000, lr=1e-05, min_images_per_class=10, num_workers=8, positive_dist_threshold=25, random_resized_crop=0.5, resume_model='/content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/default/2022-12-30_05-54-55/best_model.pth', resume_train=None, saturation=0.7, save_dir='default', seed=0, test_set_folder='/content/tokyo_xs/test', use_amp16=False)
2022-12-30 06:30:26   The outputs are being saved in logs/default/2022-12-30_06-30-26
/usr/local/lib/python3.8

In [ ]:
os.chdir('/content/drive/MyDrive/Class_Vis_Geo/CosPlace')
#  in training defalut backbone (resnet 18 was used)
# in training  fc_output_dim =512
! python3 eval.py --dataset_folder /content/tokyo_xs --backbone resnet18 \
 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/cos_place_epoch10/2023-01-12_11-37-00/best_model.pth

2023-01-13 18:36:29   eval.py --dataset_folder /content/tokyo_xs --backbone resnet18 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/cos_place_epoch10/2023-01-12_11-37-00/best_model.pth
2023-01-13 18:36:29   Arguments: Namespace(L=2, M=10, N=5, alpha=30, augmentation_device='cuda', backbone='resnet18', batch_size=32, brightness=0.7, classifiers_lr=0.01, contrast=0.7, dataset_folder='/content/tokyo_xs', device='cuda', epochs_num=50, fc_output_dim=512, groups_num=8, hue=0.5, infer_batch_size=16, iterations_per_epoch=10000, lr=1e-05, min_images_per_class=10, num_workers=8, positive_dist_threshold=25, random_resized_crop=0.5, resume_model='/content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/cos_place_epoch10/2023-01-12_11-37-00/best_model.pth', resume_train=None, saturation=0.7, save_dir='default', seed=0, test_set_folder='/content/tokyo_xs/test', use_amp16=False)
2023-01-13 18:36:29   The outputs are being saved in logs/default/2023-01-13_18-36-28
/usr

## tokyo night



In [ ]:
A = os.listdir('/content/drive/MyDrive/Class_Vis_Geo/datasets/tokyo_night')
size_tokyo_night =0 
for i in A:
  if i.split('.')[-1] =='jpg':
      size_tokyo_night +=1
print('tokyo night size is: ',size_tokyo_night )

tokyo night size is:  104


In [ ]:
shutil.rmtree('/content/tokyo_night')
if not os.path.exists('/content/tokyo_night'):
  os.makedirs('/content/tokyo_night')


In [ ]:
if not os.path.exists('/content/tokyo_night/test/queries_v1'):
  os.makedirs('/content/tokyo_night/test/queries_v1')
%cp -r /content/drive/MyDrive/Class_Vis_Geo/datasets/tokyo_night/*.jpg /content/tokyo_night/test/queries_v1

In [ ]:
if not os.path.exists('/content/tokyo_night/test/database'):
  os.makedirs('/content/tokyo_night/test/database')
%cp -r /content/tokyo_xs/test/database /content/tokyo_night/test/database

In [ ]:
# os.chdir("/content/tokyo_xs/test") 
# os.rename("tokyo_night","queries_v1")


In [ ]:
os.chdir('/content/drive/MyDrive/Class_Vis_Geo/CosPlace')
#  in training defalut backbone (resnet 18 was used)
# in training  fc_output_dim =512
! python3 eval.py --dataset_folder /content/tokyo_night --backbone resnet18 \
 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/default/2022-12-30_05-54-55/best_model.pth

2023-01-13 19:41:17   eval.py --dataset_folder /content/tokyo_night --backbone resnet18 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/default/2022-12-30_05-54-55/best_model.pth
2023-01-13 19:41:17   Arguments: Namespace(L=2, M=10, N=5, alpha=30, augmentation_device='cuda', backbone='resnet18', batch_size=32, brightness=0.7, classifiers_lr=0.01, contrast=0.7, dataset_folder='/content/tokyo_night', device='cuda', epochs_num=50, fc_output_dim=512, groups_num=8, hue=0.5, infer_batch_size=16, iterations_per_epoch=10000, lr=1e-05, min_images_per_class=10, num_workers=8, positive_dist_threshold=25, random_resized_crop=0.5, resume_model='/content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/default/2022-12-30_05-54-55/best_model.pth', resume_train=None, saturation=0.7, save_dir='default', seed=0, test_set_folder='/content/tokyo_night/test', use_amp16=False)
2023-01-13 19:41:17   The outputs are being saved in logs/default/2023-01-13_19-41-17
/usr/local/lib/

In [ ]:
# since training was paused lets check best modles from previous epoch too 
os.chdir('/content/drive/MyDrive/Class_Vis_Geo/CosPlace')
#  in training defalut backbone (resnet 18 was used)
# in training  fc_output_dim =512
! python3 eval.py --dataset_folder /content/tokyo_night --backbone resnet18 \
 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/cos_place_epoch10/2023-01-12_11-37-00/best_model.pth

2023-01-13 19:43:55   eval.py --dataset_folder /content/tokyo_night --backbone resnet18 --fc_output_dim 512 --resume_model /content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/cos_place_epoch10/2023-01-12_11-37-00/best_model.pth
2023-01-13 19:43:55   Arguments: Namespace(L=2, M=10, N=5, alpha=30, augmentation_device='cuda', backbone='resnet18', batch_size=32, brightness=0.7, classifiers_lr=0.01, contrast=0.7, dataset_folder='/content/tokyo_night', device='cuda', epochs_num=50, fc_output_dim=512, groups_num=8, hue=0.5, infer_batch_size=16, iterations_per_epoch=10000, lr=1e-05, min_images_per_class=10, num_workers=8, positive_dist_threshold=25, random_resized_crop=0.5, resume_model='/content/drive/MyDrive/Class_Vis_Geo/CosPlace/logs/cos_place_epoch10/2023-01-12_11-37-00/best_model.pth', resume_train=None, saturation=0.7, save_dir='default', seed=0, test_set_folder='/content/tokyo_night/test', use_amp16=False)
2023-01-13 19:43:55   The outputs are being saved in logs/default/2023-01-13_19-4